# **CSCE 5218 / CSCE 4930 Deep Learning**

# **HW1a The Perceptron** (20 pt)


In [1]:
# Get the datasets
!wget http://huang.eng.unt.edu/CSCE-5218/test.dat
!wget http://huang.eng.unt.edu/CSCE-5218/train.dat


--2022-01-31 19:36:55--  http://huang.eng.unt.edu/CSCE-5218/test.dat
Resolving huang.eng.unt.edu (huang.eng.unt.edu)... 129.120.123.155
Connecting to huang.eng.unt.edu (huang.eng.unt.edu)|129.120.123.155|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2844 (2.8K)
Saving to: ‘test.dat.1’

test.dat.1          100%[===================>]   2.78K  --.-KB/s    in 0s      

2022-01-31 19:36:55 (5.95 MB/s) - ‘test.dat.1’ saved [2844/2844]

--2022-01-31 19:36:55--  http://huang.eng.unt.edu/CSCE-5218/train.dat
Resolving huang.eng.unt.edu (huang.eng.unt.edu)... 129.120.123.155
Connecting to huang.eng.unt.edu (huang.eng.unt.edu)|129.120.123.155|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11244 (11K)
Saving to: ‘train.dat.1’

train.dat.1         100%[===================>]  10.98K  --.-KB/s    in 0s      

2022-01-31 19:36:55 (43.6 MB/s) - ‘train.dat.1’ saved [11244/11244]



In [2]:
# Take a peek at the datasets
!head train.dat
!head test.dat

A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13	
1	1	0	0	0	0	0	0	1	1	0	0	1	0
0	0	1	1	0	1	1	0	0	0	0	0	1	0
0	1	0	1	1	0	1	0	1	1	1	0	1	1
0	0	1	0	0	1	0	1	0	1	1	1	1	0
0	1	0	0	0	0	0	1	1	1	1	1	1	0
0	1	1	1	0	0	0	1	0	1	1	0	1	1
0	1	1	0	0	0	1	0	0	0	0	0	1	0
0	0	0	1	1	0	1	1	1	0	0	0	1	0
0	0	0	0	0	0	1	0	1	0	1	0	1	0
A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13
1	1	1	1	0	0	1	1	0	0	0	1	1	0
0	0	0	1	0	0	1	1	0	1	0	0	1	0
0	1	1	1	0	1	1	1	1	0	0	0	1	0
0	1	1	0	1	0	1	1	1	0	1	0	1	0
0	1	0	0	0	1	0	1	0	1	0	0	1	0
0	1	1	0	0	1	1	1	1	1	1	0	1	0
0	1	1	1	0	0	1	1	0	0	0	1	1	0
0	1	0	0	1	0	0	1	1	0	1	1	1	0
1	1	1	1	0	0	1	1	0	0	0	0	1	0


### Build the Perceptron Model

You will need to complete some of the function definitions below.  DO NOT import any other libraries to complete this. 

In [9]:
import math
import itertools
import re


# Corpus reader, all columns but the last one are coordinates;
#   the last column is the label
def read_data(file_name):
    # NOTE: I had to adjust the implementation of this function in order for the code below to run.
    f = open(file_name, 'r')

    data = []
    # Discard header line
    f.readline()
    for instance in f.readlines():
        if not re.search('\t', instance): continue
        # instance = [list(map(int, instance.strip().split('\t')))]
        instance = list(map(int, instance.strip().split('\t')))
        # Add a dummy input so that w0 becomes the bias
        instance = [-1] + instance
        # data += instance
        data.append(instance)
    return data


def dot_product(array1, array2):
    """
    The dot product is the sum of pairwise products between two vectors/arrays.
    :param array1: some array of (real) numerical data
    :param array2: some array of (real) numerical data
    :return: number representing the dot product of the two arrays
    """
    return sum(x*y for x,y in zip(array1, array2))


def sigmoid(x):
    """
    The sigmoid function is a smooth, nonlinear activation function that restricts the real line to a range of (0,1).
    The formula of the sigmoid function is `1/(1+math.exp(-x))`.
    :param x: some number in the real numbers
    :return: the number equal to Sigmoid(x) in the range (0,1)
    """
    return 1/(1+math.exp(-x))

# The output of the model, which for the perceptron is 
# the sigmoid function applied to the dot product of 
# the instance and the weights
def output(weight, instance):
    """
    A Perceptron is the composition of the dot product of weight and instance and sigmoid.
    :param weight: some array of weight values.
    :param instance: some array of input values
    :return:
    """
    return sigmoid(dot_product(weight, instance))

# Predict the label of an instance; this is the definition of the perceptron
# you should output 1 if the output is >= 0.5 else output 0
def predict(weights, instance):
    """
    A prediction is 1 (True) if the output of the perceptron is greater than some threshold (0.5). 0 (False) otherwise.
    :param weight: some array of weight values.
    :param instance: some array of input values
    :return: 1 or 0 depending on the value of the output
    """
    return int(output(weights, instance) >= .5)


# Accuracy = percent of correct predictions
def get_accuracy(weights, instances):
    # You do not to write code like this, but get used to it
    correct = sum([1 if predict(weights, instance) == instance[-1] else 0
                   for instance in instances])
    return correct * 100 / len(instances)


# Train a perceptron with instances and hyperparameters:
#       lr (learning rate) 
#       epochs
# The implementation comes from the definition of the perceptron
#
# Training consists on fitting the parameters which are the weights
# that's the only thing training is responsible to fit
# (recall that w0 is the bias, and w1..wn are the weights for each coordinate)
#
# Hyperparameters (lr and epochs) are given to the training algorithm
# We are updating weights in the opposite direction of the gradient of the error,
# so with a "decent" lr we are guaranteed to reduce the error after each iteration.
def train_perceptron(instances, lr, epochs):
    # NOTE: I had to adjust the implementation of this function in order for the code below to run.

    # Initialize the weights
    # weights = [0] * (len(instances[0])-1)
    weights = [0] * (len(instances[0]))

    for _ in range(epochs):
        for instance in instances:
            # Forward propagation (evaluate the model on this data point)
            in_value = dot_product(weights, instance)
            output = sigmoid(in_value)
            output = predict(weights, instance)
            error = instance[-1] - output
            # Back propagation (adjust the weights by stepping in the direction of the negative gradient
            for i in range(len(weights)):
                weights[i] += lr * error * output * (1-output) * instance[i]

    return weights

## Run it

In [11]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
# instances_tr

In [5]:
lr = 0.005
epochs = 5
weights = train_perceptron(instances_tr, lr, epochs)
accuracy = get_accuracy(weights, instances_te)
print(f"#tr: {len(instances_tr):3}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
      f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#tr: 400, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0


## Questions

Answer the following questions. Include your implementation and the output for each question.



### Question 1

In `train_perceptron(instances, lr, epochs)`, we have the following code:
```
in_value = dot_product(weights, instance)
output = sigmoid(in_value)
error = instance[-1] - output
```

Why don't we have the following code snippet instead?
```
output = predict(weights, instance)
error = instance[-1] - output
```

#### TODO Add your answer here (text only)

### Question 1 Answer
The function `predict` does more than just calculate the `dot_product` and apply `sigmoid`. It also checks if the
`output` is higher than the threshold of 0.5 and returns a `0` or `1`. Running a training loop with the second code
snippet reveals immediately that this is a problem:
```
#tr: 20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 32.0
#tr: 20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 32.0
#tr: 20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 32.0
#tr: 20, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 32.0
#tr: 20, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 32.0
```
Clearly, the modified inference that uses the prediction directly does not learn anything! This is because of the way
back-propagation is done. Consider the second code snippet with the following back-propagation step.
```
output = predict(weights, instance)
error = instance[-1] - output
# Back propagation (adjust the weights by stepping in the direction of the negative gradient
for i in range(len(weights)):
    weights[i] += lr * error * output * (1-output) * instance[i]
```
In this case `output = 1` or `output=0`. In either case the term `output * (1-output)` equals `0`. Thus, this step size
is constantly zero, and the model learns nothing each iteration. Training against non-binary outputs allows this step to
 have variable, nonzero size.


### Question 2
Train the perceptron with the following hyperparameters and calculate the accuracy with the test dataset.

```
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]              # number of epochs
lr = [0.005, 0.01, 0.05]              # learning rate
```

TODO: Write your code below and include the output at the end of each training loop (NOT AFTER EACH EPOCH)
of your code.The output should look like the following:
```
# tr:  20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
[and so on for all the combinations]
```
You will get different results with different hyperparameters.

### Question 2 Solution


In [10]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]     # number of epochs
lr_array = [0.005, 0.01, 0.05]        # learning rate

for lr in lr_array:
    for tr_size in tr_percent:
        for epochs in num_epochs:
            size =  round(len(instances_tr)*tr_size/100)
            pre_instances = instances_tr[0:size]
            weights = train_perceptron(pre_instances, lr, epochs)
            accuracy = get_accuracy(weights, instances_te)
            print(f"#tr: {len(pre_instances):0}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
                    f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#tr: 20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 32.0
#tr: 20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 32.0
#tr: 20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 32.0
#tr: 20, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 32.0
#tr: 20, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 32.0
#tr: 40, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 32.0
#tr: 40, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 32.0
#tr: 40, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 32.0
#tr: 40, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 32.0
#tr: 40, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 32.0
#tr: 100, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 32.0
#tr: 100, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 32.0
#tr: 100, epochs:  20, lea

### Question 3
Write a couple paragraphs interpreting the results with all the combinations of hyperparameters. Drawing a plot will probably help you make a point. In particular, answer the following:
- A. Do you need to train with all the training dataset to get the highest accuracy with the test dataset?
- B. How do you justify that training the second run obtains worse accuracy than the first one (despite the second one uses more training data)?
   ```
#tr: 100, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 71.0
#tr: 200, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
```
- C. Can you get higher accuracy with additional hyperparameters (higher than `80.0`)?
- D. Is it always worth training for more epochs (while keeping all other hyperparameters fixed)?

#### TODO: Add your answer here (code and text)



### Question 3 Answers
- **A. Do you need to train with all the training dataset to get the highest accuracy with the test dataset?**

No. You do not need to train with all the training dataset to get the highest accuracy.
As long as the subset of the training data is relatively representative of the entire dataset, then training will
accomplish the same result. Sometimes, when other hyperparameters are adjusted, an experiment with less data can even
achieve higher final accuracy.
Consider for example these two trials from the above set:

```
#tr: 40, epochs: 100, learning rate: 0.010; Accuracy (test, 100 instances): 80.0
#tr: 20, epochs: 100, learning rate: 0.050; Accuracy (test, 100 instances): 91.0
```

Clearly, the second trial here (with 20 data points) achieves higher accuracy than the other (with 40 data points).

- B. **How do you justify that training the second run obtains worse accuracy than the first one (despite the second one uses more training data)?**
   ```
#tr: 100, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 71.0
#tr: 200, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
```

In the second example, the learning rate is much lower. This learning rate is proportional to the "step size" in each
iteration. Thus, with a lower step size, the model will iterate through parameter space more slowly, meaning it will
find a local minimum much slower and training will require more steps to achieve the same accuracy. Since both
experiments only had 20 epochs at this measurement (and 20 epochs is not very long into training), the second experiment
would have traversed less distance, and thus is farther from a local minimum in the parameter space.

- C. **Can you get higher accuracy with additional hyperparameters (higher than `80.0`)?**
Yes. Adjusting the hyperparameters can achieve higher than 80% accuracy. Consider this experiment:
```
#tr: 400, epochs:  10, learning rate: 0.050; Accuracy (test, 100 instances): 100.0
```
Clearly, the accuracy is 100% on the sampled test data, which is higher than 80% accuracy.




- D. **Is it always worth training for more epochs (while keeping all other hyperparameters fixed)?**
Sometimes, additional training yields diminishing returns. Consider for example this training loop:
```
#tr: 20, epochs:   5, learning rate: 0.050; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  10, learning rate: 0.050; Accuracy (test, 100 instances): 74.0
#tr: 20, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 85.0
#tr: 20, epochs:  50, learning rate: 0.050; Accuracy (test, 100 instances): 91.0
#tr: 20, epochs: 100, learning rate: 0.050; Accuracy (test, 100 instances): 91.0
```
In this loop, we see that the perceptron learns quickly at first, but the rate at which accuracy improves decreases
over time. In fact, the model is no more accurate after 100 epochs than it is at 50 epochs. This means that even with
the same hyperparameters, the model converged to its final accuracy around 50 epochs. Any additional training after that
 point accomplished no improved accuracy, and thus additional training is not worth the effort.